<a href="https://colab.research.google.com/github/ampnb/eeg-meditation/blob/main/eeg_group_Morlet_wavelet_transform_10m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1tf3y7AsGztdlYR07QJM_2bQXm9OTqkCg

In [ ]:
!gdown --id 1nW4sTz3Y4L6zUAS6yM8QogN2Y4XjXBDj

In [ ]:
!unzip -o sunday-meditator.zip

In [ ]:
!unzip -o sunday-non-meditator.zip

In [ ]:
!unzip -o sunday-meditator.zip

In [ ]:
!pip install -U mne

In [ ]:
pip install mne

In [12]:
import mne
import numpy as np
import os
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

In [16]:
meditator_data_path = '/content/sunday-meditator'
non_meditator_data_path = '/content/sunday-non-meditator'

# I apply the Morlet wavelet transform within the specified frequency ranges, and calculates the mean power for each band

In [ ]:
# Define frequency bands
frequency_bands = {'Delta': (1, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (13, 30), 'Gamma': (30, 40)}

def load_and_create_epochs(data_path, epoch_length=60, overlap=0):
    """
    Load EEG data, create epochs, and return a list of Epochs objects for all files in the directory.
    """
    epochs_list = []
    for file_name in sorted(os.listdir(data_path)):
        if file_name.endswith('.fif'):
            file_path = os.path.join(data_path, file_name)
            raw = mne.io.read_raw_fif(file_path, preload=True)
            events = mne.make_fixed_length_events(raw, duration=epoch_length-overlap)
            epochs = mne.Epochs(raw, events, tmin=0, tmax=epoch_length-1/raw.info['sfreq'], baseline=None, preload=True)
            epochs_list.append(epochs)
    return epochs_list

def calculate_average_band_power(epochs_list, frequency_bands):
    """
    Calculate average band power for each frequency band across all epochs and return a dictionary.
    """
    band_powers = {band: [] for band in frequency_bands}
    for epochs in epochs_list:
        for band, (fmin, fmax) in frequency_bands.items():
            power = mne.time_frequency.tfr_morlet(epochs, freqs=np.linspace(fmin, fmax, 20), n_cycles=2, return_itc=False, average=True, decim=3)
            band_power = power.data.mean(axis=2).mean(axis=0)  # Average across time and epochs
            band_powers[band].append(band_power)
    return band_powers

def plot_band_powers(band_powers_meditator, band_powers_non_meditator, frequency_bands):
    """
    Plot the average band power for meditators vs non-meditators.
    """
    fig, axs = plt.subplots(len(frequency_bands), 1, figsize=(10, 15), sharex=True)
    if len(frequency_bands) == 1:  # Ensure axs is iterable
        axs = [axs]

    for ax, band in zip(axs, frequency_bands):
        med_mean = np.mean([np.mean(power) for power in band_powers_meditator[band]])
        non_med_mean = np.mean([np.mean(power) for power in band_powers_non_meditator[band]])
        med_std = np.std([np.mean(power) for power in band_powers_meditator[band]])
        non_med_std = np.std([np.mean(power) for power in band_powers_non_meditator[band]])

        ax.bar('Meditator', med_mean, yerr=med_std, capsize=5)
        ax.bar('Non-Meditator', non_med_mean, yerr=non_med_std, capsize=5)
        ax.set_title(f'{band} Band Power')
        ax.set_ylabel('Power')

    plt.tight_layout()
    plt.show()

# Load and create epochs
epochs_meditator = load_and_create_epochs(meditator_data_path, epoch_length=60)
epochs_non_meditator = load_and_create_epochs(non_meditator_data_path, epoch_length=60)

# Calculate average band power
band_powers_meditator = calculate_average_band_power(epochs_meditator, frequency_bands)
band_powers_non_meditator = calculate_average_band_power(epochs_non_meditator, frequency_bands)

# Plotting
plot_band_powers(band_powers_meditator, band_powers_non_meditator, frequency_bands)

In [20]:
from scipy.stats import ttest_ind

def print_statistical_summary(band_powers_meditator, band_powers_non_meditator, frequency_bands):
    """
    Perform t-tests for each frequency band and print a summary of the results.
    """
    print("Statistical Summary of Band Powers Between Meditators and Non-Meditators:\n")

    for band in frequency_bands:
        # Flatten the list of band powers for all segments for each group
        med_powers = [power for segment_powers in band_powers_meditator[band] for power in segment_powers]
        non_med_powers = [power for segment_powers in band_powers_non_meditator[band] for power in segment_powers]

        # Perform t-test
        t_stat, p_val = ttest_ind(med_powers, non_med_powers)
        significance = "Significant" if p_val < 0.05 else "Not Significant"

        print(f"{band} Band: t={t_stat:.2f}, p={p_val:.4f} ({significance})")


print_statistical_summary(band_powers_meditator, band_powers_non_meditator, frequency_bands)


Statistical Summary of Band Powers Between Meditators and Non-Meditators:

Delta Band: t=1.12, p=0.2624 (Not Significant)
Theta Band: t=2.01, p=0.0455 (Significant)
Alpha Band: t=2.25, p=0.0248 (Significant)
Beta Band: t=1.12, p=0.2639 (Not Significant)
Gamma Band: t=1.73, p=0.0843 (Not Significant)
